In [1]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
gym_data = pd.read_csv('gym_members_exercise_tracking.csv')

In [3]:
scaler = StandardScaler()

# Replace 'df' with 'gym_data' to use the correct DataFrame
scaled_features = scaler.fit_transform(gym_data[['Age', 'Height (m)', 'Weight (kg)', 'Calories_Burned', 'BMI']])
# Also adjust column names to match the DataFrame

df_scaled = pd.DataFrame(scaled_features, columns=['Age', 'Height', 'Weight', 'Calories_Burned', 'BMI'])

In [4]:
# Initialize OneHotEncoder without the 'sparse' argument and handle unknown values
encoder = OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)
# sparse_output=False to return a dense array

encoded_features = encoder.fit_transform(gym_data[['Workout_Type', 'Gender']]) # Use gym_data instead of df
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['Workout_Type', 'Gender']))

# Assuming df_scaled is defined in the previous cell, if not, please define it
df_preprocessed = pd.concat([df_scaled, encoded_df], axis=1)

In [5]:
pca = PCA(n_components=2)
pca_result = pca.fit_transform(df_preprocessed)
df_pca = pd.DataFrame(pca_result, columns=['PCA1', 'PCA2'])

In [6]:
# Choose optimal K based on Elbow and Silhouette results (e.g., K=4)
optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
df_scaled['Cluster'] = kmeans.fit_predict(df_scaled)

# Apply K-Means Clustering with k=5
kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(df_pca)
df_pca
# Label the clusters based on their characteristics
cluster_labels = {
    0: "Strength and High Intensity Workouts",
    1: "Cardio-Focused Individuals",
    2: "Weight Loss Beginners",
    3: "Endurance Booster",
    4: "Balanced Energy for Beginners"
}
gym_data['Cluster'] = clusters
gym_data['Cluster_Label'] = gym_data['Cluster'].map(cluster_labels)


In [7]:
# Save the model
import joblib
joblib.dump(kmeans, 'kmeans_model.pkl')
print("Model saved as 'kmeans_model.pkl'")

Model saved as 'kmeans_model.pkl'
